In [55]:
import random
import string
import numpy as np
import pandas as pd
from tabulate import tabulate

### Config

In [31]:
DISH_COUNT = 300
MIN_CALORIES_PER_DISH = 100
MAX_CALORIES_PER_DISH = 1200

DAYS_PER_MENU = 7
DISHES_PER_DAY = 3

POPULATION_SIZE = 20
MATING_POPULATION_SIZE = 10
ITERATIONS = 1000

CALORIES_PER_DAY = 2000
REAL_DATA = True

#### Helper functions

In [58]:
def visualize_menu(menu):
    print("Score: " + str(score_menu(menu)))
    headers = ["Day", "Calories", "Dishes"]
    calories = count_calories_per_day(menu)
    table = []
    for day in range(DAYS_PER_MENU):
        table.append([day, calories[day], menu[day]])
    print(tabulate(table, headers, tablefmt="fancy_grid"))

#### Test data

In [59]:
if not REAL_DATA:
    dishes = {}
    for _ in range(DISH_COUNT):
        name = ''.join(random.choices(string.ascii_lowercase, k=6))
        dishes[name] = random.randint(MIN_CALORIES_PER_DISH, MAX_CALORIES_PER_DISH)
    dishes = list(dishes.items())

    print(*dishes[:7], sep='n')

#### Real data

In [60]:
if REAL_DATA:
    dishes = {}
    data = pd.read_excel('menu_database.xlsx')
    df = pd.DataFrame(data, columns=['name', 'Food Group', "Calories"])
    while len(dishes) != DISH_COUNT:
        random_choice = df.sample()
        if MIN_CALORIES_PER_DISH < random_choice.Calories.values[0] < MAX_CALORIES_PER_DISH:
            dishes[random_choice.name.values[0]] = random_choice.Calories.values[0]
    dishes = list(dishes.items())
    print(*dishes[:7], sep='\n')

('Dried Sweetened Mango', 319.0)
('Squab (Pigeon) Light Meat Without Skin Raw', 134.0)
('Beef Round Top Round Steak Boneless Separable Lean And Fat Trimmed To 0 Inch Fat Select Raw', 120.0)
('Rice White Short-Grain Raw Unenriched', 358.0)
('Milk And Cereal Bar', 413.0)
('Pretzels Soft Gluten Free', 297.0)
('Snacks Corn Cakes', 387.0)


#### Sample random menu

In [61]:
def create_random_menu():
    menu = np.ndarray((7, 3), tuple)
    for day in range(DAYS_PER_MENU):
        for dish_index in range(DISHES_PER_DAY):
            menu[day, dish_index] = random.choice(dishes)
    return menu

In [62]:
def count_calories_per_day(menu):
    calories = []
    for day in range(DAYS_PER_MENU):
        calories.append(0)
        for dish_index in range(DISHES_PER_DAY):
            calories[-1] += menu[day, dish_index][1]
    return calories


def score_menu(menu):
    differences = [calories - CALORIES_PER_DAY for calories in count_calories_per_day(menu)]
    squares = [calories ** 2 for calories in differences]
    return sum(squares)

In [63]:
sample_menu = create_random_menu()
visualize_menu(sample_menu)

Score: 12468607.0
╒═══════╤════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│   Day │   Calories │ Dishes                                                                                                       │
╞═══════╪════════════╪══════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│     0 │        399 │ [('Squid Baked Or Broiled Fat Not Added In Cooking', 113.0)                                                  │
│       │            │  ('Turkey Dark Meat Roasted Skin Not Eaten', 167.0)                                                          │
│       │            │  ('Milk Shakes Thick Chocolate', 119.0)]                                                                     │
├───────┼────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│     1 │        656 │ [('Pork Fresh Variety

In [69]:
def crossover(first_menu, second_menu):
    child = first_menu.copy()
    for day_index in range(DAYS_PER_MENU):
        if random.choice((0, 1)) == 1:
            child[day_index] = second_menu[day_index]
    return child


def random_mutate(menu):
    menu[random.randint(0, DAYS_PER_MENU - 1), random.randint(0, DISHES_PER_DAY - 1)] = random.choice(dishes)
    return menu


def swap_mutate(menu):
    first_day_index = random.randint(0, DAYS_PER_MENU - 1)
    second_day_index = random.randint(0, DAYS_PER_MENU - 1)
    dish_index = random.randint(0, DISHES_PER_DAY - 1)
    menu[first_day_index, dish_index], menu[second_day_index, dish_index] = menu[second_day_index, dish_index], menu[
        first_day_index, dish_index]
    return menu

In [70]:
population = []
for _ in range(POPULATION_SIZE):
    population.append(create_random_menu())

In [73]:
for i in range(ITERATIONS):
    population.sort(key=score_menu)
    mating_population = population[:MATING_POPULATION_SIZE]

    offspring_population = []
    for _ in range(POPULATION_SIZE - MATING_POPULATION_SIZE):
        offspring_population.append(crossover(random.choice(mating_population), random.choice(mating_population)))
        if random.random() < 0.1:
            offspring_population[-1] = random_mutate(offspring_population[-1])
        if random.random() < 0.1:
            offspring_population[-1] = swap_mutate(offspring_population[-1])

    population = mating_population + offspring_population

In [74]:
population.sort(key=score_menu)
visualize_menu(population[0])

Score: 612.0
╒═══════╤════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│   Day │   Calories │ Dishes                                                                                                                         │
╞═══════╪════════════╪════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│     0 │       2000 │ [('Fat Back Cooked', 750.0)                                                                                                    │
│       │            │  ('Pretzels Soft Ready-To-Eat Unsalted Buttered', 350.0)                                                                       │
│       │            │  ('Fat Goose', 900.0)]                                                                                                         │
├───────┼────────────┼─────────────────────────────────────────────────────